# **FLARE** 
 FLARE (Forward-Looking Active Retrieval Augmented Generation)

* **Deciding when to retrieve** — They claim that LLMs should retrieve only when they are uncertain in their prediction. Assuming LLMs are well calibrated, meaning that the probability they predict, is actually how unsure they are about the prediction, they adopt an active retrieval strategy to retrieve only when the generated tokens have a low probability.
* **What to retrieve** — They say it is important to consider what LMs intend to generate in the future, as the goal of active retrieval is to benefit future generations. Therefore, they propose anticipating the future by generating a temporary next sentence, using it as a query to retrieve relevant documents, and then regenerating the next sentence conditioning on the retrieved documents.

## Installing all the Libraries

In [1]:
!pip install openai==0.28
!pip install lancedb
!pip install langchain==0.0.354
!pip install openai
!pip install tiktoken
!pip install sentence_transformers
!pip install arxiv
!pip install pymupdf
!pip install gradio
!pip install ArxivLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: p

### Import the libraries

In [2]:
import langchain
from langchain.document_loaders import ArxivLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import FlareChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Uncomment the below if you want to see all the intermediate steps
# langchain.verbose=True

from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from io import BytesIO
from langchain.document_loaders import PyPDFLoader
import langchain
from langchain.document_loaders import ArxivLoader
import gradio as gr
import lancedb
from langchain.vectorstores import LanceDB
from langchain.document_loaders import ArxivLoader
from langchain.chains import FlareChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from langchain.llms import OpenAI
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-youropenaiapikey"

### Instantiate LLM

In [ ]:
llm = OpenAI()

### Initialize the Embeddings model

In [ ]:
model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

### Load the data

In [ ]:
# fetch docs from arxiv, in this case it's the FLARE paper
docs = ArxivLoader(query="2305.06983", load_max_docs=2).load()


# here is example https://arxiv.org/pdf/2305.06983.pdf
# you need to pass this number to query 2305.06983
# fetch docs from arxiv, in this case it's the FLARE paper
docs = ArxivLoader(query="2305.06983", load_max_docs=1).load()

### Text splitter

In [ ]:
# instantiate text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

# split the document into chunks
doc_chunks = text_splitter.split_documents(docs)

### Instantiate the Database

In [ ]:
# lancedb
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "documentsai",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)
vector_store = LanceDB.from_documents(doc_chunks, embeddings, connection=table)

vector_store_retriever = vector_store.as_retriever()

### Define the FLare Chain

In [ ]:
flare = FlareChain.from_llm(
    llm=llm, retriever=vector_store_retriever, max_generation_len=300, min_prob=0.45
)

### Create a gradio generate function for interactive UI

In [5]:
# Define a function to generate FLARE output based on user input
def generate_flare_output(input_text):
    output = flare.run(input_text)
    return output


input = gr.Text(
    label="Prompt",
    show_label=False,
    max_lines=1,
    placeholder="Enter your prompt",
    container=False,
)

iface = gr.Interface(
    fn=generate_flare_output,
    inputs=input,
    outputs="text",
    title="My AI bot",
    description="FLARE implementation with lancedb & bge embedding.",
)


iface.launch(debug=True, share=True)

(…)a171121fb03f394dc42974275/.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

(…)fb03f394dc42974275/1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

(…)b775ca171121fb03f394dc42974275/README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

(…)75ca171121fb03f394dc42974275/config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

(…)974275/config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

(…)fb03f394dc42974275/tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

(…)b775ca171121fb03f394dc42974275/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)5ca171121fb03f394dc42974275/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3611dcbabf749dee48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3611dcbabf749dee48.gradio.live


Thanks